This test case attempts to search for ghosts by comparing an LSST catalog with an external truth catalog over the same area and verifying that all of the LSST sources within the truth catalog's detection limit correspond to sources in the truth catalog.

Comparing one HSC catalog to Gaia DR2 demonstrates that this test may not be adequate, as there are dozens of HSC sources that do not appear in Gaia (though not much effort was put into making sure that we could compare Gaia and HSC magnitudes).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import os
import json
import numpy as np
import pandas
data_dir = '/project/danielsf'
valid_visit_path = os.path.join(data_dir, 'valid_hsc_visits.txt')
gaia_overlap_pickle_path = os.path.join(data_dir, 'gaia_hsc_overlap_pandas_with_photometry.pickle')
assert os.path.isfile(valid_visit_path)
assert os.path.isfile(gaia_overlap_pickle_path)

In [ ]:
from lsst.daf.persistence import Butler
repo_dir = '/datasets/hsc/repo/rerun/DM-13666/WIDE'
butler = Butler(repo_dir)

Assemble a list of valid HSC WIDE dataIds

In [ ]:
valid_data_id = []
with open(valid_visit_path, 'r') as in_file:
    for line in in_file:
        data_id = json.loads(line)
        valid_data_id.append(data_id)
data_id = valid_data_id[10]

In [ ]:
gaia_df = pandas.read_pickle(gaia_overlap_pickle_path)

In [ ]:
hsc_src = butler.get('src', dataId=valid_data_id[10])

find rough RA, Dec bounds of HSC region and get the Gaia overlap therewith

In [ ]:
ra_min = np.degrees(hsc_src['coord_ra'].min())
ra_max = np.degrees(hsc_src['coord_ra'].max())
dec_min = np.degrees(hsc_src['coord_dec'].min())
dec_max = np.degrees(hsc_src['coord_dec'].max())

In [ ]:
gaia_df_overlap = gaia_df[(gaia_df.ra<=ra_max) & (gaia_df.ra>=ra_min)
                          & (gaia_df.dec<=dec_max) & (gaia_df.dec>=dec_min)]

Find HSC sources that are brighter than the dimmest Gaia magnitude

In [ ]:
photo_calib = butler.get('calexp_photoCalib', dataId=data_id)

In [ ]:
hsc_mag = photo_calib.instFluxToMagnitude(hsc_src, 'base_PsfFlux')

In [ ]:
max_gaia_mag = gaia_df_overlap.phot_g_mean_mag.max()

In [ ]:
mag_cut_dexes = np.where(hsc_mag[:,0]<=max_gaia_mag)

spatially associate Gaia and HSC sources

In [ ]:
import scipy.spatial as scipy_spatial

In [ ]:
normalized_gaia_ra = gaia_df_overlap.ra.values*np.cos(np.radians(gaia_df_overlap.dec.values))
gaia_spatial_data = np.array([normalized_gaia_ra, gaia_df_overlap.dec.values]).transpose()

In [ ]:
normalized_hsc_ra = np.degrees(hsc_src['coord_ra'][mag_cut_dexes])*np.cos(hsc_src['coord_dec'][mag_cut_dexes])
hsc_dec = np.degrees(hsc_src['coord_dec'][mag_cut_dexes])
hsc_spatial_data = np.array([normalized_hsc_ra, hsc_dec]).transpose()

In [ ]:
%time
gaia_tree = scipy_spatial.cKDTree(gaia_spatial_data, leafsize=4)

In [ ]:
%time
hsc_dist, hsc_dex = gaia_tree.query(hsc_spatial_data)

In [ ]:
%time
hsc_tree = scipy_spatial.cKDTree(hsc_spatial_data, leafsize=4)

In [ ]:
%time
gaia_dist, gaia_dex = hsc_tree.query(gaia_spatial_data)

In [ ]:
calexp = butler.get('calexp', dataId=data_id)

In [ ]:
wcs = calexp.getWcs()

Plot images and sources.  Gaia sources in the field are yellow circles.  HSC sources that are (probably?) bright enough to be detected by Gaia are cyan crosses.

In [ ]:
import lsst.geom as lsst_geom

In [ ]:
gaia_sky_pts = [lsst_geom.SpherePoint(rr, dd, lsst_geom.degrees)
                for rr, dd in zip(gaia_df_overlap.ra.values, gaia_df_overlap.dec.values)]

In [ ]:
gaia_pixel_points = wcs.skyToPixel(gaia_sky_pts)

In [ ]:
gaia_pixel_points[70]

In [ ]:
gaia_x = [p.getX() for p in gaia_pixel_points]
gaia_y = [p.getY() for p in gaia_pixel_points]

In [ ]:
hsc_sky_points = [lsst_geom.SpherePoint(rr, dd, lsst_geom.radians)
                  for rr, dd in zip(hsc_src['coord_ra'][mag_cut_dexes],
                                    hsc_src['coord_dec'][mag_cut_dexes])]

In [ ]:
hsc_pixel_points = wcs.skyToPixel(hsc_sky_points)
hsc_x = [p.getX() for p in hsc_pixel_points]
hsc_y = [p.getY() for p in hsc_pixel_points]

In [ ]:
import lsst.afw.display as afwDisplay

In [ ]:
plt.rcParams['figure.figsize'] = (22.0, 22.0) 
plt.rcParams['lines.markeredgewidth'] = 2.0
plt.rcParams['lines.markerfacecolor'] = 'none'
plt.rcParams['lines.markersize'] = 12.0
plt.figure()
afwDisplay.setDefaultBackend('matplotlib')
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(calexp)
plt.plot(gaia_x, gaia_y, color='y', alpha=1.0,
        marker='o', linestyle='none')
plt.plot(hsc_x, hsc_y, color='c', marker='+', linestyle='none')
#plt.ylim(2000,2500)
#plt.xlim(0,250)
plt.show()

Note source in pixel ~ (250, 2100); Gaia finds something that HSC does not

### Everything below here is junk

In [ ]:
plt.rcParams.keys()

In [ ]:
my_channel = '{}_test_channel'.format(os.environ['USER'])
server = 'https://lsst-lsp-stable.ncsa.illinois.edu'

In [ ]:
ff='{}/firefly/slate.html?__wsch={}'.format(server, my_channel)
IFrame(ff,800,600)

In [ ]:
afwDisplay.setDefaultBackend('firefly')
afw_display = afwDisplay.getDisplay(frame=0, 
                                    host=server,
                                    name=my_channel)

In [ ]:
calexp = butler.get('calexp', dataId=data_id)

In [ ]:
plt.rcParams['figure.figsize'] = (22.0, 22.0) 
plt.figure()
afwDisplay.setDefaultBackend('matplotlib')
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(calexp)
plt.show()

In [ ]:
data_id

In [ ]:
wcs = calexp.getWcs()

In [ ]:
help(wcs.skyToPixel)

In [ ]:
plt.figure()
plt.plot([1,2,3],[4,5,6])